In [1]:
from rag_eval import ChunkEvalBaseBuilder
from fcgb.cfg.precompiled import get_llm, get_checkpointer

In [2]:
base_builder = ChunkEvalBaseBuilder(
    llm=get_llm(llm_model='google'),
    output_path='../data',
    builder_config={
        'max_pages': 15,
        'eval_batch_size': 8
    },
    prompts_config={
        'path': None,
        'random_queries': 'random_queries',
        'paper_queries': 'paper_queries',
        'keyword_cloud': 'keyword_cloud',
        'chunk_eval_system': 'chunk_eval_system',
        'chunk_eval_task': 'chunk_eval_task',
        'doc_context_system': 'doc_context_system',
        'doc_context_update': 'doc_context_update',
        'doc_context_aggregation': 'doc_context_aggregation'
    },
    doc_search_config={
        'main_queries_num': 4,
        'paper_queries_num': 5,
        'max_results': 5,
    },
    chunk_eval_config={
        'chunk_size': 600,
        'chunk_overlap': 0,
        'max_queries': 5,
        'context_agg_interval': 5
    },
    memory=get_checkpointer(checkpointer_mode='local')
)

In [9]:
base_builder.extend_docs(target_docs=150)

Current docs: 93, Target docs: 150, Turns needed: 3


Searching for papers with query: landscape of type II string theory compactifications with fluxes
Searching for papers with query: human-robot interaction in collaborative manufacturing environments
Searching for papers with query: ab-initio calculations of nuclear structure far from stability
Searching for papers with query: impact of climate change on alpine plant communities
Searching for papers with query: stochastic gyrokinetic simulations tokamak transport
Searching for papers with query: magnetospheric ULF waves ionospheric effects modeling
Searching for papers with query: conformal field theory entanglement entropy black holes
Searching for papers with query: perovskite solar cells stability enhancement strategies
Searching for papers with query: metamaterial-based perfect absorbers for enhanced terahertz imaging
Searching for papers with query: causal relationships between extreme weather events and socioeconomic inequality
Searching for papers with query: impact of air pollut

In [10]:
base_builder._remove_oversized_docs()

Removing oversized docs: ['0106155v2', '2505.08012v1', '0711.1500v1', '0906.0429v1', '2110.07217v2', '2108.03383v2', '2312.08640v1', '1208.5869v2', '0904.4601v2', '2409.04657v2', '2101.10252v2', '2502.13746v1', '2506.03611v1', '2310.08843v1', '2007.06720v1', '2207.09863v1', '1811.02816v1', '2202.11336v1', '2202.08188v1', '2409.05168v1', '1910.13501v1', '2011.11127v1', '2407.09632v2', '2503.18399v1', '1408.3711v1', '2112.10858v3', '1409.6103v1', '1911.12257v3', '2409.12649v1', '1708.07283v1', '2308.03828v2', '1010.3994v1', '2001.01041v1', '2412.19863v1', '1309.7385v1', '0801.0584v2', '0509003v3', '2304.06445v1']
Removed 38 oversized docs. Remaining docs: 115


In [15]:
base_builder.summary()

All docs: 115
Docs to evaluate: 15 - 13.04% of all docs
Docs evaluated: 100 - 86.96% of all docs
New docs per turn: 20
Evaluations per turn: 8


In [14]:
await base_builder.evaluate_docs(target_docs=100)
# 44 minutes for 75 docs

Removing oversized docs: []
Removed 0 oversized docs. Remaining docs: 115


Evaluating docs:   0%|          | 0/1 [00:00<?, ?it/s, Evaluated=99, Target Docs=100]

2110.03478v2.json:   0%|          | 0/60 [00:00<?, ?it/s, Negative=0, Positive=0]

Saving evaluation results on document ../data/docs/2110.03478v2.pdf...
